###### References: 
- https://docs.python.org/3/library/asyncio.html
- Fluent Python, 2nd Edition, by Luciano Ramalho. Chapter 23: Attribute Descriptors

# Attribute Descriptors

A descriptor is a class that implements a dynamic protocol consisting of `__get__`, `__set__`, and `__delete__` methods.  
The `property` class implements the full descriptor protocol.

## Descriptor Example: Attribute  Validation

### LineItem #3: A Simple Descriptor


<img src="Descriptor.png" width="75%">

<img src="MNG.png" width="75%">

In [1]:
class Quantity:  # Descriptor is a protocol-based feature, no sub-classing required

    def __init__(self, storage_name):
        self.storage_name = storage_name 

    def __set__(self, instance, value):  # called when there is an attempt to assign to managed attribute
        if value > 0:
            instance.__dict__[self.storage_name] = value  # store in dict
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)


In [2]:
class LineItem:
    weight = Quantity('weight')  # <1>
    price = Quantity('price')  # <2>

    def __init__(self, description, weight, price):  # <3>
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [3]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.weight, raisins.description, raisins.price

(10, 'Golden raisins', 6.95)

In [4]:
raisins.subtotal()

69.5

In [5]:
raisins.weight  = -20

ValueError: weight must be > 0

In [6]:
raisins.weight

10

## LineItem Take #4: Automatic Naming of Storage Attributes

In [7]:
class Quantity:

    def __set_name__(self, owner, name):  # self is the descriptor instance, owner is the managed class, name of attribute
        self.storage_name = name

    def __set__(self, instance, value):   # <3>
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)
            
class LineItem:
    weight = Quantity()  # No need to pass managed name
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [8]:
import model_v4c as model

In [9]:
class LineItem:
    weight = model.Quantity()  # put model quantity to use
    price = model.Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

## LineItem Take #5: A New Descriptor Type

### NonBlank descriptor type

In [10]:
import abc

In [11]:
class Validated(abc.ABC):

    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        value = self.validate(self.storage_name, value)  # delegates validation
        instance.__dict__[self.storage_name] = value  # uses return value

    @abc.abstractmethod
    def validate(self, name, value):  # abstract method
        """return validated value or raise ValueError"""

In [12]:
class Quantity(Validated):
    """a number greater than zero"""

    def validate(self, name, value):  # implementation of template method
        if value <= 0:
            raise ValueError(f'{name} must be > 0')
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""

    def validate(self, name, value):
        value = value.strip()
        if not value:  # if nothing is left, reject value
            raise ValueError(f'{name} cannot be blank')
        return value  # template give opportunity to clean up, convert or normalize the data received.

In [13]:
class LineItem:
    description = NonBlank()  # <2>
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [14]:
br_nuts = LineItem('Brazil Nuts', 10, 34.95)

In [15]:
br_nuts.description = ' '

ValueError: description cannot be blank

In [16]:
void = LineItem('', 1, 1)

ValueError: description cannot be blank

#  Overriding versus Nonoverriding Descriptors

In [17]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return f'<{cls_name(obj)} object>'

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print(f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')


### essential classes for this example ###

class Overriding:  # <1>
    """a.k.a. data descriptor or enforced descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)  # <2>

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class OverridingNoGet:  # <3>
    """an overriding descriptor without ``__get__``"""

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class NonOverriding:  # <4>
    """a.k.a. non-data or shadowable descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)


class Managed:  # <5>
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):  # <6>
        print(f'-> Managed.spam({display(self)})')

## Overriding Descriptor

In [18]:
obj = Managed()

# retrives the descriptor instance from the class
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [19]:
# same if retrived from managed class
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [20]:
# invokes the __set__ descriptor method
obj.over = 4

-> Overriding.__set__(<Overriding object>, <Managed object>, 4)


In [21]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [22]:
obj.__dict__['over'] = 8
(vars(obj))

{'over': 8}

In [23]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


## Overriding Descriptor Without `__get__`

In [24]:
obj.over_no_get

In [25]:
Managed.over_no_get

In [26]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [27]:
obj.over_no_get

In [28]:
obj.__dict__['over_no_get'] = 9
obj.over_no_get

9

In [29]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [30]:
obj.over_no_get

9

## Nonoverriding Descriptor

In [31]:
obj = Managed()

In [32]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [33]:
obj.non_over = 7

In [34]:
obj.non_over

7

In [35]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [36]:
del obj.non_over

In [37]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


## Overwriting a Descriptor in the Class

No descriptor type survives being overwritten on the class itself:


In [38]:
obj = Managed() 

In [39]:
Managed.over = 1

In [40]:
Managed.over_no_get = 2

In [41]:
Managed.non_over = 3

In [42]:
obj.over, obj.over_no_get, obj.non_over

(1, 2, 3)

# Methods are Descriptors

### Methods are non-overriding descriptors:

In [43]:
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x1098e0e80>>

In [44]:
Managed.spam

<function __main__.Managed.spam(self)>

In [45]:
obj.spam()

-> Managed.spam(<Managed object>)


In [46]:
Managed.spam()

TypeError: Managed.spam() missing 1 required positional argument: 'self'

In [47]:
Managed.spam(obj)

-> Managed.spam(<Managed object>)


In [48]:
obj.spam.__func__ is Managed.spam

True

In [49]:
obj.spam  = 7
obj.spam

7

-

In [50]:
import collections

In [51]:
class Text(collections.UserString):

    def __repr__(self):
        return 'Text({!r})'.format(self.data)

    def reverse(self):
        return self[::-1]

In [52]:
word = Text('forward')
word

Text('forward')

In [53]:
word.reverse()

Text('drawrof')

In [54]:
Text.reverse(Text('backward'))

Text('drawkcab')

In [55]:
type(Text.reverse), type(word.reverse)

(function, method)

In [56]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')])) 

['diaper', (30, 20, 10), Text('desserts')]

In [57]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

In [58]:
Text.reverse.__get__(None, Text)

<function __main__.Text.reverse(self)>

In [59]:
word.reverse

<bound method Text.reverse of Text('forward')>

In [60]:
word.reverse.__self__

Text('forward')

In [61]:
word.reverse.__func__ is Text.reverse

True

# Descriptor Usage Tips

* Use `property` to keep it simple
* Read-only descriptors require `__set__`
* Validation descriptors can  work with `__set__` only
* Caching can be done efficiently with `__get__` only
* Nonspecial methods can be shadowed by instance attributes, unlike special methods

# Descriptor Docstring and Overriding Deletion

The docstring of a descriptor class is used to document every instance of descritor in the managed class.